# Debugging Script 
This is a manual script of the functions for debugging purposes

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import os
import pyodbc
import glob
import copy
import PySimpleGUI as sg
import traceback

# Bring in Part 1 Code

In [2]:
!pip -q install -r ./config_files/requirements.txt
import pandas as pd
pd.options.mode.chained_assignment = None

^C


In [ ]:
# Bring in functions
%run ./scripts/Part_1.ipynb

 # Individual Files

In [9]:
# inputs 
ds_ID = 'DS45'

comparison_processed_data vs comparison_no_geozone_df:
- comparison_processed_data has columns appending the DSID and has the geozone at the end (jurisdiction level)
- features are all of teh columns


In [10]:
# mgra_ind
comparison_processed_data, comparison_no_geozone_df, features = download_DS_data(ds_ID, jur_level)

DS45's jurisdiction did not merge correctly.


In [6]:
# CPA Level Aggregation
comparison_processed_data_cpa = cpa_aggregation_ind(comparison_no_geozone_df, cpa_level)

In [7]:
# Jurisdiction aggregation
comparison_processed_data_jur = jur_aggregation_ind(comparison_no_geozone_df, jur_level)

In [8]:
# Region Aggregation
comparison_processed_data_reg = region_aggregation_ind(comparison_no_geozone_df)

# Comparison Files

In [9]:
# Inputs for comparison 
first_ID = "DS38"
second_ID = 'DS41'

In [10]:
# Download the necessary data
# First ID
comparison_first_ID_processed_data, comparison_first_ID_no_geozone_df, first_ID_features = download_DS_data(first_ID, jur_level)

# Second ID
comparison_second_ID_processed_data, comparison_second_ID_no_geozone_df, second_ID_features = download_DS_data(second_ID, jur_level)

DS38's jurisdiction did not merge correctly.
DS41's jurisdiction did not merge correctly.


## Both Files

In [11]:
# MGRA_Both Files
# This does the right thing and drops 'secondary' not sure how it knows to do that but it does 
mgra_both = concat_dfs_temp(comparison_first_ID_no_geozone_df, comparison_second_ID_no_geozone_df)

In [12]:
# CPA both
cpa_both = cpa_aggregation(comparison_first_ID_no_geozone_df, comparison_second_ID_no_geozone_df, cpa_level)

In [13]:
# Jur both 
jur_both = jur_aggregation(comparison_first_ID_no_geozone_df, comparison_second_ID_no_geozone_df, jur_level)

In [14]:
# Region Both
region_both = region_aggregation(comparison_first_ID_no_geozone_df, comparison_second_ID_no_geozone_df)

## Diff Files

In [15]:
'''The issue is that in mgra_both 'secondary' does not exist '''

'''The issue with this code originally is that the input originally was create_diff(features_first_ID, features_second_ID, first_second_ID_comparison) and the features_first_ID had non numeric columns such as secondary and elementary while mgra-both only had numeric (because of the groupby at the end) so the way the code was written it was trying to subtract columns that were alike in 'features_first_ID' and 'features_second_ID' but some didn't exist in mgra_both'''

'''Create diff goes and keeps things in the same order, it doesn't use set. Now the code is grabbing columns from the both file itself. '''

mgra_diff = create_diff(list([x[:-5] for x in mgra_both if x[-5:] == '_' + first_ID]), list([x[:-5] for x in mgra_both if x[-5:] == '_' + second_ID]), mgra_both)

C:\Users\cra\AppData\Local\Temp\ipykernel_20132\564207998.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']


In [16]:
cpa_diff = create_diff_temp(list([x[:-5] for x in cpa_both if x[-5:] == '_' + first_ID]),list([x[:-5] for x in cpa_both if x[-5:] == '_' + second_ID]), cpa_both) 

C:\Users\cra\AppData\Local\Temp\ipykernel_20132\3044449395.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']


In [17]:
jur_diff = create_diff_temp(list([x[:-5] for x in jur_both if x[-5:] == '_' + first_ID]), list([x[:-5] for x in jur_both if x[-5:] == '_' + second_ID]), jur_both)

C:\Users\cra\AppData\Local\Temp\ipykernel_20132\3044449395.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']


In [18]:
region_diff = create_diff_temp(list([x[:-5] for x in region_both if x[-5:] == '_' + first_ID]), list([x[:-5] for x in region_both if x[-5:] == '_' + first_ID]), region_both)

C:\Users\cra\AppData\Local\Temp\ipykernel_20132\3044449395.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']


# Creating outputs for J Drive

In [13]:
for DSID in ['DS42']:
    # MGRA LEVEL
    comparison_processed_data, comparison_no_geozone_df, features = download_DS_data(DSID, jur_level)
    print(f"{DSID} mgra level output created")

    # Luz
    luz_data = comparison_no_geozone_df.drop('mgra', axis=1).groupby(['luz_id', 'year']).sum()

    # Region
    region_data = comparison_no_geozone_df.drop(['mgra', 'luz_id'], axis=1).groupby(['year']).sum()

    # Send to proper folder 
    level = 'mgra'
    comparison_no_geozone_df.to_csv(f'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/{level}_{DSID}_ind_QA.csv', index=False)
    print(f"mgra {DSID} is completed and in J drive")

    level = 'luz'
    luz_data.to_csv(f'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/{level}_{DSID}_ind_QA.csv', index=True)
    print(f"LUZ {DSID} is completed and in J drive")

    level = 'region'
    region_data.to_csv(f'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/{level}_{DSID}_ind_QA.csv', index=True)
    print(f"Region {DSID} is completed and in J drive")


DS99's jurisdiction did not merge correctly.
DS99 mgra level output created
mgra DS99 is completed and in J drive
LUZ DS99 is completed and in J drive
Region DS99 is completed and in J drive


In [6]:
for DSID in ['DS35', 'DS39', 'DS41', 'DS38', 'DS42', 'DS99']:
    # MGRA LEVEL
    comparison_processed_data, comparison_no_geozone_df, features = download_DS_data(DSID, jur_level)
    print(f"{DSID} mgra level output created")

    # Luz
    #luz_data = comparison_no_geozone_df.drop('mgra', axis=1).groupby(['luz_id', 'year']).sum()
    
    # Taz
    taz_data = comparison_no_geozone_df.drop('mgra', axis=1).groupby(['taz', 'year']).sum()

    # Region
    #region_data = comparison_no_geozone_df.drop(['mgra', 'luz_id'], axis=1).groupby(['year']).sum()

    # Send to proper folder 
    #level = 'mgra'
    # comparison_no_geozone_df.to_csv(f'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/{level}_{DSID}_ind_QA.csv', index=False)
    # print(f"mgra {DSID} is completed and in J drive")

    # level = 'luz'
    # luz_data.to_csv(f'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/{level}_{DSID}_ind_QA.csv', index=True)
    # print(f"LUZ {DSID} is completed and in J drive")

    level = 'taz'
    taz_data.to_csv(f'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/{level}_{DSID}_ind_QA.csv', index=True)
    print(f"TAZ {DSID} is completed and in J drive")

    # level = 'region'
    # region_data.to_csv(f'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/{level}_{DSID}_ind_QA.csv', index=True)
    # print(f"Region {DSID} is completed and in J drive")


DS35's jurisdiction did not merge correctly.
DS35 mgra level output created
TAZ DS35 is completed and in J drive
DS39's jurisdiction did not merge correctly.
DS39 mgra level output created
TAZ DS39 is completed and in J drive
DS41's jurisdiction did not merge correctly.
DS41 mgra level output created
TAZ DS41 is completed and in J drive
DS38's jurisdiction did not merge correctly.
DS38 mgra level output created
TAZ DS38 is completed and in J drive
DS42's jurisdiction did not merge correctly.
DS42 mgra level output created
TAZ DS42 is completed and in J drive
DS99's jurisdiction did not merge correctly.
DS99 mgra level output created
TAZ DS99 is completed and in J drive


# Creating Outputs for SQL Data

In [ ]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=demographic_warehouse;'
                    'Trusted_Connection=yes;')

query_all = '''SELECT *
  FROM [isam].[xpef40].[abm_mgra13_based_input_np]'''

sql_query = pd.read_sql_query(query_all,conn)
sql_df_all = pd.DataFrame(sql_query)                   


mgra_output = sql_df_all.groupby(['yr', 'mgra']).sum().reset_index()

mgra_output = mgra_output.rename(columns={"yr": "year"})

region_output = mgra_output.groupby('year').sum()
region_output = region_output.reset_index()
region_output

# region_output.to_csv(f'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/region_SQLDS99_ind_QA.csv', index=False)

# MGRA series 15 outputs

In [16]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=SQL2014B8.sandag.org;'
                      'Database=GeoDepot;'
                      'Trusted_Connection=yes;')

In [17]:
query_all = '''SELECT
      [MGRA]
	  ,cpa_data.NAME AS CPA
	  ,jur_data.Name AS Jurisdiction
      ,[LUZ]
  FROM [GeoDepot].[gis].[MGRA15] as mgra_base
  LEFT JOIN [GeoDepot].[gis].[CITIES] AS jur_data
  ON mgra_base.City = jur_data.City
  LEFT JOIN [GeoDepot].[gis].[CITYCPA] AS cpa_data
  ON mgra_base.CPA = cpa_data.CPA'''

In [18]:
sql_query = pd.read_sql_query(query_all,conn)
sql_df_all = pd.DataFrame(sql_query)

In [19]:
sql_df_all

,MGRA,CPA,Jurisdiction,LUZ
0,1,Mid-City:Eastern Area,San Diego,10
1,2,Downtown,San Diego,28
2,3,None,El Cajon,112
3,4,None,Encinitas,151
4,5,None,Encinitas,151
...,...,...,...,...
24346,24317,None,Unincorporated,11
24347,24318,None,Unincorporated,133
24348,24319,None,Unincorporated,225
24349,24320,None,Unincorporated,224


In [40]:
# Bringing in the data 
mgra_15_forecast_data = pd.read_csv('T:/socioec/Current_Projects/XPEF39/abm_csv_mgra15/mgra15_based_input2019_rev.csv')

In [41]:
mgra_level_data = mgra_15_forecast_data.merge(sql_df_all, how='left', left_on='mgra', right_on='MGRA')

In [50]:
mgra_level_data.groupby('LUZ').sum().drop(['mgra', 'MGRA'], axis=1)

,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,i1,i2,...,midpriceroom,upscaleroom,hotelroomtotal,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres
LUZ,,,,,,,,,,,,,,,,,,,,,
1,4619,2595,2024,0,4443,2517,1926,0,408,448,...,0,0,3118.0,17928,189,2457,0.0,0.0,0.0,0.0
2,10918,2846,8072,0,10404,2727,7677,0,934,1215,...,0,0,482.0,23754,247,3443,0.0,0.0,0.0,0.0
3,11215,2827,8388,0,10862,2746,8116,0,978,955,...,0,0,1928.0,13689,146,2139,0.0,0.0,0.0,0.0
4,22506,7569,14937,0,21934,7391,14543,0,2043,2723,...,0,0,292.0,24295,262,3578,0.0,0.0,0.0,0.0
5,17339,2910,14429,0,16537,2836,13701,0,1578,1367,...,0,0,2322.0,12996,155,1264,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2205,1825,349,31,2125,1759,335,31,77,178,...,0,0,44.0,1276,54,162,0.0,0.0,0.0,0.0
242,941,44,897,0,917,43,874,0,207,121,...,0,0,0.0,771,14,110,0.0,0.0,0.0,0.0
243,356,0,356,0,351,0,351,0,69,45,...,0,0,0.0,657,12,54,0.0,0.0,0.0,0.0


In [39]:
# MGRA level output 
mgra_level_data = mgra_15_forecast_data.groupby(['mgra']).sum()
mgra_level_data

,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,i1,i2,...,midpriceroom,upscaleroom,hotelroomtotal,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres
mgra,,,,,,,,,,,,,,,,,,,,,
1,176,110,66,0,172,107,65,0,29,24,...,0,0,0.0,87,1,27,0.0,0.0,0.0,0.0
2,56,0,56,0,52,0,52,0,9,8,...,0,0,0.0,103,1,27,0.0,0.0,0.0,0.0
3,200,23,177,0,195,23,172,0,14,40,...,0,0,0.0,142,1,27,0.0,0.0,0.0,0.0
4,4,4,0,0,2,2,0,0,2,0,...,0,0,0.0,221,1,27,0.0,0.0,0.0,0.0
5,43,43,0,0,39,39,0,0,1,3,...,0,0,0.0,221,1,27,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24317,2,2,0,0,2,2,0,0,1,1,...,0,0,0.0,137,1,11,0.0,0.0,0.0,0.0
24318,46,46,0,0,41,41,0,0,0,2,...,0,0,0.0,170,1,1,0.0,0.0,0.0,0.0
24319,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,210,1,14,0.0,0.0,0.0,0.0


In [57]:
mgra_level_data['Region'] = 'San Diego'

In [61]:
mgra_level_data.drop(['mgra', 'MGRA', 'luz_id', 'LUZ'], axis = 1).groupby('Region').sum()

,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,i1,i2,...,luxuryroom,midpriceroom,upscaleroom,hotelroomtotal,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres
Region,,,,,,,,,,,,,,,,,,,,,
San Diego,1204471,723514,438921,42036,1153032,694623,418294,40115,102683,125820,...,0,0,0,63564.0,24351,222799,0.0,0.0,0.0,0.0


In [55]:
mgra_level_data.groupby('Jurisdiction').sum().drop(['mgra', 'MGRA'], axis=1)

,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,i1,i2,...,upscaleroom,hotelroomtotal,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,LUZ
Jurisdiction,,,,,,,,,,,,,,,,,,,,,
Carlsbad,46376,31775,13284,1317,43833,30078,12560,1195,2153,3329,...,0,4836.0,19724,758,4420,0.0,0.0,0.0,0.0,125029
Chula Vista,84102,50998,29249,3855,82551,49962,28778,3811,7192,8709,...,0,1920.0,130162,1075,12506,0.0,0.0,0.0,0.0,103616
Coronado,9588,5482,4106,0,7509,4457,3052,0,539,478,...,0,1717.0,29397,223,3272,0.0,0.0,0.0,0.0,21511
Del Mar,2613,1889,724,0,2083,1506,577,0,112,179,...,0,350.0,2244,66,1008,0.0,0.0,0.0,0.0,2901
El Cajon,36056,15280,18581,2195,34997,14785,18098,2114,4815,5234,...,0,898.0,83625,588,4690,0.0,0.0,0.0,0.0,71995
Encinitas,26189,21370,4080,739,24680,20135,3852,693,1670,1942,...,0,624.0,84624,821,4617,0.0,0.0,0.0,0.0,122756
Escondido,48473,28005,16730,3738,47315,27309,16353,3653,4864,6385,...,0,912.0,35613,940,9399,0.0,0.0,0.0,0.0,189526
Imperial Beach,9980,5797,3998,185,9577,5549,3851,177,1237,1470,...,0,78.0,19519,152,2130,0.0,0.0,0.0,0.0,25362
La Mesa,25763,13939,11521,303,25138,13596,11252,290,2890,3165,...,0,543.0,64192,469,4413,0.0,0.0,0.0,0.0,46881


In [62]:
mgra_level_data.drop(['mgra', 'MGRA', 'luz_id', 'LUZ'], axis = 1).groupby('Region').sum().to_csv(f'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_15_2019_CSV_outputs/region_2019_CSV_Data_ind_QA.csv', index=True)

In [29]:
mgra_level_data = mgra_level_data.reset_index()
mgra_level_data

,mgra,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,i1,...,midpriceroom,upscaleroom,hotelroomtotal,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres
0,1,176,110,66,0,172,107,65,0,29,...,0,0,0.0,87,1,27,0.0,0.0,0.0,0.0
1,2,56,0,56,0,52,0,52,0,9,...,0,0,0.0,103,1,27,0.0,0.0,0.0,0.0
2,3,200,23,177,0,195,23,172,0,14,...,0,0,0.0,142,1,27,0.0,0.0,0.0,0.0
3,4,4,4,0,0,2,2,0,0,2,...,0,0,0.0,221,1,27,0.0,0.0,0.0,0.0
4,5,43,43,0,0,39,39,0,0,1,...,0,0,0.0,221,1,27,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24316,24317,2,2,0,0,2,2,0,0,1,...,0,0,0.0,137,1,11,0.0,0.0,0.0,0.0
24317,24318,46,46,0,0,41,41,0,0,0,...,0,0,0.0,170,1,1,0.0,0.0,0.0,0.0
24318,24319,0,0,0,0,0,0,0,0,0,...,0,0,0.0,210,1,14,0.0,0.0,0.0,0.0
24319,24320,0,0,0,0,0,0,0,0,0,...,0,0,0.0,205,1,1,0.0,0.0,0.0,0.0


In [33]:
mgra_level_data = mgra_level_data.merge(sql_df_all, how='left', left_on='mgra', right_on='MGRA')

In [37]:
mgra_level_data['lUZ']

KeyError: 'lUZ'

In [35]:
mgra_level_data['luz_id'] == mgra_level_data['LUZ']

KeyError: 'LUZ'

In [38]:
mgra_level_data.columns

Index(['mgra', 'hs', 'hs_sf', 'hs_mf', 'hs_mh', 'hh', 'hh_sf', 'hh_mf',
       'hh_mh', 'i1',
       ...
       'effective_acres', 'land_acres', 'MGRA_x', 'CPA_x', 'Jurisdiction_x',
       'LUZ_x', 'MGRA_y', 'CPA_y', 'Jurisdiction_y', 'LUZ_y'],
      dtype='object', length=112)

# Comparing CSV totals with what is in database
Numbers look off. 

In [57]:
# Localise with . files 
if os.getcwd().split('\\')[-1] == 'scripts':
    config_filename = '../config_files/ds_config.yml'
else:
    config_filename = './config_files/ds_config.yml'

In [58]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [59]:
def csv_hhp_total_check(DSID):
    output_df = pd.DataFrame(index= list(config[DSID].keys())[:-1], columns = ['hhp_sum_'+DSID])

    for ind_csv in output_df.index:
        temp_df = pd.read_csv(config[DSID][ind_csv])
        sum_value = temp_df['hhp'].sum()
        output_df['hhp_sum_'+DSID][ind_csv] = sum_value
        

    return output_df

In [60]:
ds35_output = csv_hhp_total_check('DS35')
ds38_output = csv_hhp_total_check('DS38')
ds41_output = csv_hhp_total_check('DS41')
ds42_output = csv_hhp_total_check('DS42')

merged_DSIDs = pd.DataFrame(index = [2016, 2018, 2020, 2023, 2025, 2026, 2029, 2030, 2032, 2035, 2040, 2045, 2050])

merged_DSIDs['DS35'] = list(ds35_output['hhp_sum_DS35'])
merged_DSIDs['DS38'] = list(ds38_output['hhp_sum_DS38'])
merged_DSIDs['DS41'] = list(ds41_output['hhp_sum_DS41'])
merged_DSIDs['DS42'] = list(ds42_output['hhp_sum_DS42'])

merged_DSIDs

,DS35,DS38,DS41,DS42
2016,3203210,3203210,3203210,3203210
2018,3237268,3237268,3237268,3237268
2020,3263592,3263592,3263592,3263592
2023,3309666,3309666,3309666,3309666
2025,3345075,3345075,3345075,3345075
2026,3362198,3362198,3362198,3362198
2029,3410379,3410379,3410379,3410379
2030,3425282,3425282,3425282,3425282
2032,3453451,3453451,3453451,3453451
2035,3492730,3492730,3492730,3492730


In [50]:
merged_DSIDs = pd.DataFrame(index = [2016, 2018, 2020, 2023, 2025, 2026, 2029, 2030, 2032, 2035, 2040, 2045, 2050])
merged_DSIDs

""
2016
2018
2020
2023
2025
2026
2029
2030
2032
2035


In [54]:
list(ds35_output['hhp_sum_DS35'])

[3203210,
 3237268,
 3263592,
 3309666,
 3345075,
 3362198,
 3410379,
 3425282,
 3453451,
 3492730,
 3548558,
 3592070,
 3618458]

In [56]:
merged_DSIDs = pd.DataFrame(index = [2016, 2018, 2020, 2023, 2025, 2026, 2029, 2030, 2032, 2035, 2040, 2045, 2050])

merged_DSIDs['DS35'] = list(ds35_output['hhp_sum_DS35'])
merged_DSIDs['DS38'] = list(ds38_output['hhp_sum_DS38'])
merged_DSIDs['DS41'] = list(ds41_output['hhp_sum_DS41'])
merged_DSIDs['DS42'] = list(ds42_output['hhp_sum_DS42'])


,DS35,DS38,DS41,DS42
2016,3203210,3203210,3203210,3203210
2018,3237268,3237268,3237268,3237268
2020,3263592,3263592,3263592,3263592
2023,3309666,3309666,3309666,3309666
2025,3345075,3345075,3345075,3345075
2026,3362198,3362198,3362198,3362198
2029,3410379,3410379,3410379,3410379
2030,3425282,3425282,3425282,3425282
2032,3453451,3453451,3453451,3453451
2035,3492730,3492730,3492730,3492730


In [47]:
ds35_output

,hhp_sum_DS35
DS35_2016,3203210
DS35_2018,3237268
DS35_2020,3263592
DS35_2023,3309666
DS35_2025,3345075
DS35_2026,3362198
DS35_2029,3410379
DS35_2030,3425282
DS35_2032,3453451
DS35_2035,3492730
